In [1]:
import pandas as pd
import unittest
from enum import Enum

In [2]:
class Result(Enum):
    '''
    Enum with two possible states: WIN and LOSS
    '''
    WIN = 1
    LOSS = 0

In [3]:
class TestPokerHand(unittest.TestCase):
    '''
    Class that implements the unit tests
    '''

    def test_comparison(self):
        self.assertTrue(PokerHand("TC TH 5C 5H KH").compare_with(PokerHand("9C 9H 5C 5H AC")) == Result.WIN)
        self.assertTrue(PokerHand("TS TD KC JC 7C").compare_with(PokerHand("JS JC AS KC TD")) == Result.LOSS)
        self.assertTrue(PokerHand("7H 7C QC JS TS").compare_with(PokerHand("7D 7C JS TS 6D")) == Result.WIN)
        self.assertTrue(PokerHand("5S 5D 8C 7S 6H").compare_with(PokerHand("7D 7S 5S 5D JS")) == Result.LOSS)
        self.assertTrue(PokerHand("AS AD KD 7C 3D").compare_with(PokerHand("AD AH KD 7C 4S")) == Result.LOSS)
        self.assertTrue(PokerHand("TS JS QS KS AS").compare_with(PokerHand("AC AH AS AS KS")) == Result.WIN)
        self.assertTrue(PokerHand("TS JS QS KS AS").compare_with(PokerHand("TC JS QC KS AC")) == Result.WIN)
        self.assertTrue(PokerHand("TS JS QS KS AS").compare_with(PokerHand("QH QS QC AS 8H")) == Result.WIN)
        self.assertTrue(PokerHand("AC AH AS AS KS").compare_with(PokerHand("TC JS QC KS AC")) == Result.WIN)
        self.assertTrue(PokerHand("AC AH AS AS KS").compare_with(PokerHand("QH QS QC AS 8H")) == Result.WIN)
        self.assertTrue(PokerHand("TC JS QC KS AC").compare_with(PokerHand("QH QS QC AS 8H")) == Result.WIN)
        self.assertTrue(PokerHand("7H 8H 9H TH JH").compare_with(PokerHand("JH JC JS JD TH")) == Result.WIN)
        self.assertTrue(PokerHand("7H 8H 9H TH JH").compare_with(PokerHand("4H 5H 9H TH JH")) == Result.WIN)
        self.assertTrue(PokerHand("7H 8H 9H TH JH").compare_with(PokerHand("7C 8S 9H TH JH")) == Result.WIN)
        self.assertTrue(PokerHand("7H 8H 9H TH JH").compare_with(PokerHand("TS TH TD JH JD")) == Result.WIN)
        self.assertTrue(PokerHand("7H 8H 9H TH JH").compare_with(PokerHand("JH JD TH TC 4C")) == Result.WIN)
        self.assertTrue(PokerHand("JH JC JS JD TH").compare_with(PokerHand("4H 5H 9H TH JH")) == Result.WIN)
        self.assertTrue(PokerHand("JH JC JS JD TH").compare_with(PokerHand("7C 8S 9H TH JH")) == Result.WIN)
        self.assertTrue(PokerHand("JH JC JS JD TH").compare_with(PokerHand("TS TH TD JH JD")) == Result.WIN)
        self.assertTrue(PokerHand("JH JC JS JD TH").compare_with(PokerHand("JH JD TH TC 4C")) == Result.WIN)
        self.assertTrue(PokerHand("4H 5H 9H TH JH").compare_with(PokerHand("7C 8S 9H TH JH")) == Result.WIN)
        self.assertTrue(PokerHand("4H 5H 9H TH JH").compare_with(PokerHand("TS TH TD JH JD")) == Result.LOSS)
        self.assertTrue(PokerHand("4H 5H 9H TH JH").compare_with(PokerHand("JH JD TH TC 4C")) == Result.WIN)
        self.assertTrue(PokerHand("7C 8S 9H TH JH").compare_with(PokerHand("TS TH TD JH JD")) == Result.LOSS)
        self.assertTrue(PokerHand("7C 8S 9H TH JH").compare_with(PokerHand("JH JD TH TC 4C")) == Result.WIN)
        self.assertTrue(PokerHand("TS TH TD JH JD").compare_with(PokerHand("JH JD TH TC 4C")) == Result.WIN)

In [4]:
class PokerHandRater():
    '''
    Class with functions to rate the poker hand
    
    Attributes
    ----------
    card_denom_mapping : {string:string}
        Mapping of denominations with its current numeric value
    '''
    
    card_denom_mapping = {
        'T': '10',
        'J': '11',
        'Q': '12',
        'K': '13',
        'A': '14'
    }
    
    def replace_card_denom(self, card_denom):
        # Replace the denominations T,J,Q,K,A with
        # its numeric value and convert the denominations
        # to integer
        if card_denom in self.card_denom_mapping:
            card_denom = self.card_denom_mapping[card_denom]
        return int(card_denom)
    
    def in_denomination_sequence(self, denominations):
        # Sort by denomination and calculate the difference 
        # between each denomination with the previous one 
        # to see if the cards are in denomination sequence
        denominations = denominations.sort_values()
        denom_diff = denominations.diff()[1:].unique()
        
        # If 1 is the only difference value, 
        # the cards are in denomination sequence
        in_sequence = denom_diff.shape[0] == 1 and denom_diff[0] == 1
        
        return in_sequence
    
    def rate_poker_hand(self, hand):
        # Split the hand in [denomination, suit] card pairs
        arr_hand = [[self.replace_card_denom(card[0]), card[1]] 
                    for card in hand.split()]
        # Convert the pairs to a dataframe to ease its use
        df = pd.DataFrame(arr_hand, columns=['denomination','suit'])
        
        # Group the cards by suit
        group_suit = df.groupby('suit').count()

        # If there is only one group then the five cards 
        # belong to an unique suit (it is a Flush)
        if group_suit.shape[0] == 1:
            # If the cards are in denomination sequence
            if self.in_denomination_sequence(df['denomination']):
                # If the max card is A(14) 
                # then we have a ROYAL STRAIGHT FLUSH
                if df['denomination'].max() == 14:
                    rating=10
                # If the max card is different than A 
                # then we have a STRAIGHT FLUSH
                else:
                    rating=9
            # The cards are not in denomination sequence 
            # so we have a FLUSH 
            else:
                rating=6
        # If there is more than one suit group
        else:
            # Group by denomination
            group_denom = df.groupby('denomination').count()
            
            # If there are two groups
            if group_denom.shape[0] == 2:
                # If the biggest group have 4 cards then
                # we have FOUR OF A KIND
                if int(group_denom.max()) == 4:
                    rating=8
                # If not, the only other possible distribution 
                # for having two groups is to have 3 cards in a group 
                # and 2 in another group, so we have a FULL HOUSE 
                else:
                    rating=7
            # If there are three groups
            elif group_denom.shape[0] == 3:
                # If the biggest group have 3 cards 
                # then we have THREE OF A KIND
                if int(group_denom.max()) == 3:
                    rating=4
                # If not, the only other possible distribution
                # for having three groups is to have two groups
                # with 2 cards and one group with 1 card,
                # so we have a TWO PAIR
                else:
                    rating=3
            # If there are four groups, we have ONE PAIR
            elif group_denom.shape[0] == 4:
                rating=2
            # If there are five groups
            else:
                # If the cards are in denomination sequence,
                # we have a STRAIGHT
                if self.in_denomination_sequence(df['denomination']):
                    rating=5
                # If not, we have a HIGH CARD
                else:
                    rating=1
        
        # Get the denominations of the hand sorted descending
        # to use it as a tie breaker
        tie_breaker = df['denomination'].sort_values(ascending=False)
        
        return rating, tie_breaker

In [5]:
class PokerHand():
    '''
    Class that represents a poker hand
    and can compare to another poker hand
    
    Attributes
    ----------
    
    hand : string
        A string representation of the hand
    rating: int
        The rating that the hand have
    tie_breaker: array-like, dtype=int, shape = 5
        The denominations of the hand sorted descending
        to use as a tie breaker
    '''
    def __init__(self, hand):
        self.hand = hand
        self.rating, self.tie_breaker = PokerHandRater().rate_poker_hand(hand)
    
    def compare_with(self, otherHand):
        # If the rating of this hand is greater than
        # the other hand then result=WIN
        if self.rating > otherHand.rating:  
            return Result.WIN
        # If the rating of this hand is less than
        # the other hand then result=LOSS
        elif self.rating < otherHand.rating:
            return Result.LOSS
        # If both ratings are the same, 
        # then use the tie breakers
        else:
            # As both tie breakers are sorted descending,
            # compare each pair of elements until the
            # values are different and one is greater than the other
            for i in range(len(self.tie_breaker)):
                # If the value of this hand is greater than
                # the other then result=WIN
                if self.tie_breaker[i] > otherHand.tie_breaker[i]:
                    return Result.WIN
                # If the value of this hand is less than
                # the other then result=LOSS
                elif self.tie_breaker[i] < otherHand.tie_breaker[i]:
                    return Result.LOSS

In [6]:
# Unit tests
TestPokerHand().test_comparison()